## Exampletary Notebook using LLAMA2-chat with vLLM

In [ ]:
# NOTE
# to use this repository the fewshot examples must be added to medicaNLP.bronco.fewshots

In [ ]:
from typing import List, Optional, Tuple
import itertools
import json

from datasets import load_dataset, Dataset
from vllm import LLM, SamplingParams

from nervaluate import Evaluator

from medicaNLP.bronco.bronco_utils import bronco_llama2_fewshot_prompt, parse_output
from medicaNLP.bronco.fewshots import LLAMA2_FEWSHOT_PROMPT
from medicaNLP.instructionTuning.evaluate import evaluate_bronco_prediction

In [ ]:
# load test data
test_data_instruct = load_dataset(
    'json',
    data_files={'test': "/path/to/instr_test.json"}
)['test']

### output generation

In [ ]:
# init model
llm = LLM("meta-llama/Llama-2-7b-chat-hf")
sampling_params = SamplingParams(temperature=0.1, max_tokens=240)

In [ ]:
prompts = [bronco_llama2_fewshot_prompt(p) for p in test_data_instruct]
len(prompts)

In [ ]:
## run text generation
test_outputs = llm.generate(prompts, sampling_params)

In [ ]:
# collect sentences with output from model
test_output_sentence = [
    {
        "id": x['id'],
        "text": x['input'],
        "output": output.outputs[0].text
    } for x, output in zip(test_data_instruct, test_outputs)
]
len(test_output_sentence)

### evaluation

In [ ]:
# parse output string and transform to dictionary
for pred in pred_data['predictions']:
    pred['parsed_output'] = parse_output(pred['output'], ['Medikamente', 'Diagnose', 'Behandlung'])

In [ ]:
# evaluate generated response
res, res_per_tag = evaluate_bronco_prediction(
    test_path='/data/German-medical-texts/BRONCO150/instruction_data/BRONCO_instr_test.json',
    pred_data=pred_data
)